In [ ]:
import sys
sys.path.append("../")

In [ ]:
import pandas as pd
import datetime as dt
import plotly.graph_objects as go
from technicals.indicators import RSI
from technicals.patterns import apply_patterns
from plotting import CandlePlot

In [ ]:
df_raw = pd.read_pickle("../data/EUR_USD_H1.pkl")

In [ ]:
df_raw.shape

In [ ]:
df_an = df_raw.copy()#df_raw.iloc[-6000:].copy()
df_an.reset_index(drop=True, inplace=True)

In [ ]:
df_an.shape

In [ ]:
df_an = RSI(df_an)

In [ ]:
df_an.tail()

In [ ]:
df_an = apply_patterns(df_an)

In [ ]:
df_an['EMA_200'] = df_an.mid_c.ewm(span=200, min_periods=200).mean()

In [ ]:
df_an.columns

In [ ]:
our_cols = ['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c',
                    'bid_c',   'ask_c',
             'ENGULFING', 'direction', 'EMA_200', 'RSI_14' ]

In [ ]:
df_slim = df_an[our_cols].copy()
df_slim['spread'] = df_slim.ask_c - df_slim.bid_c
df_slim.dropna(inplace=True)
df_slim.reset_index(drop=True, inplace=True)

In [ ]:
df_slim.head()

In [ ]:
BUY = 1
SELL = -1
NONE = 0
RSI_LIMIT = 50.0

def apply_signal(row):
    if row.ENGULFING == True:
        if row.direction == BUY and row.mid_l > row.EMA_200:
            if row.RSI_14 > RSI_LIMIT:
                return BUY
        if row.direction == SELL and row.mid_h < row.EMA_200:
            if row.RSI_14 < RSI_LIMIT:
                return SELL
    return NONE        

In [ ]:
df_slim["SIGNAL"] = df_slim.apply(apply_signal, axis=1)

In [ ]:
df_slim["SIGNAL"].value_counts()

In [ ]:
LOSS_FACTOR = -1.0
PROFIT_FACTOR = 1.5

def apply_take_profit(row):
    if row.SIGNAL != NONE:
        return (row.mid_c - row.mid_o) * 1.5 + row.mid_c
    else:
        return 0.0

def apply_stop_loss(row):
    if row.SIGNAL != NONE:
        return row.mid_o
    else:
        return 0.0

In [ ]:
df_slim["TP"] = df_slim.apply(apply_take_profit, axis=1)
df_slim["SL"] = df_slim.apply(apply_stop_loss, axis=1)

In [ ]:
df_slim[df_slim.SIGNAL==BUY].head()

In [ ]:
df_plot = df_slim.iloc[70:100]
cp = CandlePlot(df_plot, candles=True)

trades = cp.df_plot[cp.df_plot.SIGNAL != NONE]

markers = ['mid_c', 'TP', 'SL']
marker_colors = ['#0000FF', '#00FF00', '#FF0000']

for i in range(3):
    cp.fig.add_trace(go.Scatter(
        x = trades.sTime,
        y = trades[markers[i]],
        mode = 'markers',
        marker=dict(color=marker_colors[i], size=12)
    ))

cp.show_plot(line_traces=["EMA_200"], sec_traces=['RSI_14'], height=250)

In [ ]:
class Trade:
    def __init__(self, row):
        self.running = True
        self.start_index = row.name
        self.start_price = row.mid_c
        self.trigger_price = row.mid_c
        self.SIGNAL = row.SIGNAL
        self.TP = row.TP
        self.SL = row.SL
        self.result = 0.0
        self.end_time = row.time
        self.start_time = row.time
        self.duration = 0
        
    def close_trade(self, row, result, trigger_price):
        
        if result == PROFIT_FACTOR:
            win_amount = abs(self.start_price-self.TP)
            actual_amount = win_amount - row.spread
            result = PROFIT_FACTOR * (actual_amount / win_amount)
        else:
            loss_amount = abs(self.start_price-self.SL)
            actual_amount = loss_amount + row.spread
            result = LOSS_FACTOR * (loss_amount / actual_amount)
        
        self.running = False
        self.result = result
        self.end_time = row.time
        self.trigger_price = trigger_price
        
    def update(self, row):
        self.duration += 1
        if self.SIGNAL == BUY:
            if row.mid_h >= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.mid_h)
            elif row.mid_l <= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.mid_l)
        if self.SIGNAL == SELL:
            if row.mid_l <= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.mid_l)
            elif row.mid_h >= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.mid_h)    

In [ ]:
open_trades = []
closed_trades = []

for index, row in df_slim.iterrows():
    for ot in open_trades:
        ot.update(row)
        if ot.running == False:
            closed_trades.append(ot)
    open_trades = [x for x in open_trades if x.running == True]
    
    if row.SIGNAL != NONE:
        open_trades.append(Trade(row))    

In [ ]:
df_results = pd.DataFrame.from_dict([vars(x) for x in closed_trades])

In [ ]:
df_results.result.sum()

In [ ]:
df_results.sort_values(by="start_index", inplace=True)

In [ ]:
df_results.head()